In [1]:
# !pip install openpyxl

In [10]:
import json
import sqlite3
import pandas as pd

from datetime import datetime
from pytz import timezone

In [12]:
now_str = datetime.now(timezone('Europe/Helsinki')).strftime('%Y%m%d-%H%M%S')
now_str

'20220321-162225'

In [3]:
con = sqlite3.connect('../results/crawling.sqlite')

In [4]:
def flatten_content(x):
    return {k: v for d in x for k, v in list(d.items())} if x else {}


def has_fibre(x):
    xs = str(x).lower()
    xo = str(x)
    d = {}
    # Either substring in string
    yes_words = [
        'valokuitu', 'talokuitu', 'kuituliit', 'kuituvaraus', 'kuitukaap',
        'kuituyhteys', 'kuituverk', 'kiinteistökuit', 'kuituvalmiu',
        'kuitunett', 'kuitu asen', 'kuituiitt', 'talokuitu', 'talo kuitu'
    ]
    for w in yes_words:
        if w in xs:
            fibre_idx = xs.find(w)
            d['evidence'] = xo[fibre_idx - 50:fibre_idx + 50]
            d['has_fibre'] = 'yes'
            return d
    # Either kuitu in tietoliikenne
    not_words = [
        'kuitubitu', 'lasikuitu', 'kuituvalo', 'puu-kuitu', 'puukuitu',
        'kuitusemen', 'kuituelemen', 'kuitunen', 'kuituvalais',
        'kuitutapet', 'kuitulevy', 'kuitukatu', 'kuituprojekt'
    ]
    if 'kuitu' in xs and not any(w in xs for w in not_words):
        fibre_idx = xs.find('kuitu')
        d['evidence'] = xo[fibre_idx - 50:fibre_idx + 50]
        d['has_fibre'] = 'likely'
        return d
    elif 'kuitu' in xs:
        fibre_idx = xs.find('kuitu')
        d['evidence'] = xo[fibre_idx - 50:fibre_idx + 50]
        d['has_fibre'] = 'unlikely'
        return d
    else:
        d['evidence'] = None
        d['has_fibre'] = 'no'
        return d

In [5]:
listings = (
    pd.read_sql('SELECT * FROM LISTINGS', con)
    .assign(content = lambda x:
        x['content'].apply(lambda x: json.loads(x) if x else None))
    .assign(has_fibre = lambda x: x['content'].apply(has_fibre))
    .pipe(lambda df:
        df
        .drop('has_fibre', axis=1)
        .join(df['has_fibre'].apply(pd.Series))
    )
    .pipe(lambda df:
        df.join(pd.DataFrame.from_records(
            df['content'].apply(flatten_content).values))
    )
)

In [21]:
fibre = (
    listings
    .assign(address = lambda x: x['title'].apply(lambda x:
            x.split('m²')[-1].split(',')[0] if pd.notnull(x) else None))
    .assign(postal_code = lambda x: x['title'].apply(lambda x:
            x.split('m²')[-1].split(',')[-1].strip().split(' ')[0]
            if pd.notnull(x) else None))
    .assign(postal_area = lambda x: x['title'].apply(lambda x:
            x.split('m²')[-1].split(',')[-1].strip().split(' ')[1]
            if pd.notnull(x) else None))
    .query('address.notna()')
    [['has_fibre', 'address', 'postal_code', 'postal_area', 'evidence',
      'url']]
    .sort_values(by=['postal_code', 'address'])
)
len(fibre.query('has_fibre == "yes"'))

3062

In [14]:
fibre.to_excel(f'../results/fibre-availability-test-{now_str}.xlsx', index=False)

In [20]:
coop = (
    listings
    .assign(address = lambda x: x['title'].apply(lambda x:
            x.split('m²')[-1].split(',')[0] if pd.notnull(x) else None))
    .assign(postal_code = lambda x: x['title'].apply(lambda x:
            x.split('m²')[-1].split(',')[-1].strip().split(' ')[0]
            if pd.notnull(x) else None))
    .assign(postal_area = lambda x: x['title'].apply(lambda x:
            x.split('m²')[-1].split(',')[-1].strip().split(' ')[1]
            if pd.notnull(x) else None))
    .rename(columns={
        'Taloyhtiön nimi': 'coop'
    })
    .query('address.notna() and coop.notna()')
    [['address', 'postal_code', 'postal_area', 'coop', 'url']]
)
len(coop)

35834

In [22]:
coop.to_excel(f'../results/fibre-coop-test-{now_str}.xlsx', index=False)